In [15]:
import laspy
import pandas as pd
import rerun as rr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [2]:
laz_file_path = "whm_01_tid.laz"

In [42]:
# Open the LAZ file and extract attributes
with laspy.open(laz_file_path) as las:
    points = las.read()
    attributes = {dimension.name: getattr(points, dimension.name) for dimension in points.point_format}
    df_pcd = pd.DataFrame(attributes)

# Drop irrelevant columns
df_pcd = df_pcd[['X', 'Y', 'Z', 'red', 'green', 'blue', 'nir', 'ndvi', 'norm_g', 'mtvi2', 'tree_id']]

# Convert LAS scaled integer coordinates to floating-point coordinates
scale = las.header.scales
offset = las.header.offsets
df_pcd['X'] = df_pcd['X'] * scale[0] + offset[0]
df_pcd['Y'] = df_pcd['Y'] * scale[1] + offset[1]
df_pcd['Z'] = df_pcd['Z'] * scale[2] + offset[2]

# Show dataset info
df_pcd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222529 entries, 0 to 222528
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   X        222529 non-null  float64
 1   Y        222529 non-null  float64
 2   Z        222529 non-null  float64
 3   red      222529 non-null  uint16 
 4   green    222529 non-null  uint16 
 5   blue     222529 non-null  uint16 
 6   nir      222529 non-null  uint16 
 7   ndvi     222529 non-null  float32
 8   norm_g   222529 non-null  float32
 9   mtvi2    222529 non-null  float32
 10  tree_id  222529 non-null  int32  
dtypes: float32(3), float64(3), int32(1), uint16(4)
memory usage: 10.2 MB


In [40]:
df_tree_summary = df_pcd.groupby('tree_id').agg({
    'Z': ['mean', 'min', 'max', 'median'],
    'ndvi': 'mean',
    'mtvi2': 'mean'
}).reset_index()

df_tree_summary.columns = ['tree_id', 'Z_mean', 'Z_min', 'Z_max', 'Z_median', 'ndvi_mean', 'mtvi2_mean']
df_tree_summary.info()

available_tree_ids = df_pcd['tree_id'].unique().tolist()
print("Available tree IDs:", np.sort(available_tree_ids))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tree_id     26 non-null     int32  
 1   Z_mean      26 non-null     float64
 2   Z_min       26 non-null     float64
 3   Z_max       26 non-null     float64
 4   Z_median    26 non-null     float64
 5   ndvi_mean   26 non-null     float32
 6   mtvi2_mean  26 non-null     float32
dtypes: float32(2), float64(4), int32(1)
memory usage: 1.2 KB
Available tree IDs: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24]


In [47]:
# Normalize feature values to [0,1]
def normalize_array(arr):
    arr = np.nan_to_num(arr, nan=np.nanmin(arr))  # Replace NaNs with min value
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr) + 1e-10)  # Avoid division by zero

# Apply colormap to normalized values
def apply_colormap(values, colormap_name="viridis"):
    cmap = plt.get_cmap(colormap_name)
    return cmap(values)[:, :3]  # Extract RGB (ignore alpha)

def normalize_rgb(rgb_array):
    """Normalize 16-bit RGB values to 8-bit (0-255) without modifying the DataFrame."""
    return (rgb_array / rgb_array.max(axis=0) * 255).astype(np.uint8)

In [ ]:
# Main function to add a new visualization timestamp
def show_tree_feature(df_pcd: pd.DataFrame, tree_id: int, feature_name='RGB', colormap_name="viridis"):
    df_tree = df_pcd[df_pcd['tree_id'] == tree_id].copy()

    if df_tree.empty:
        print(f"No points found for tree_id={tree_id}.")
        return
    
    if feature_name != 'RGB' and not feature_name in df_pcd.columns:
        print(f"Feature '{feature_name}' not found in DataFrame.")
        return

    # Connect to Rerun viewer
    rr.init(f"tree_{tree_id}", spawn=False)
    rr.connect_tcp("127.0.0.1:9876")  # Use default Rerun port

    # Create a new time sequence for this feature
    rr.set_time_sequence(str(feature_name), 0)

    if feature_name == 'RGB':
        rgb_values = df_pcd[['red', 'green', 'blue']].values
        feature_colors = normalize_rgb(rgb_values)
    else:
        feature_values = df_tree[feature_name].values
        normalized_values = normalize_array(feature_values)
        feature_colors = (apply_colormap(normalized_values, colormap_name) * 255).astype(np.uint8)

    # if feature_name == 'rgb':
    #     # Extract and normalize RGB values without modifying df_pcd
    #     rgb_values = df_pcd[['red', 'green', 'blue']].values
    #     normalized_rgb = normalize_rgb(rgb_values)
    #     # Log the point cloud with original color
    #     rr.log(
    #         f"RGB",  
    #         rr.Points3D(
    #             positions=df_tree[['X', 'Y', 'Z']].values,
    #             colors=normalized_rgb
    #         )
    #     )
    # else:    
    #     # Normalize feature values and apply colormap
    #     feature_values = df_tree[feature_name].values
    #     normalized_values = normalize_array(feature_values)
    #     feature_colors = (apply_colormap(normalized_values, colormap_name) * 255).astype(np.uint8)

    #     # Log points with feature-based colors
    rr.log(
        str(feature_name),  
        rr.Points3D(
            positions=df_tree[['X', 'Y', 'Z']].values,
            colors=feature_colors
        )
    )

    print(f"Added visualization for feature:\t{feature_name}")


In [57]:
tree_id = 7

show_tree_feature(df_pcd, tree_id)

Feature 'RGB' not found in DataFrame.
